# Chapter 15
# Pitfalls of Hypothesis Testing     

## Introduction  


Data scientists are often faced with large complex datasets. These dataset typically contain a large number of variables. The **high-dimensional** nature of the data leads to some commonly encountered pitfalls which can lead to incorrect inferences.   

In this chapter we explore two important concepts related to the misuse of p-vlaues and hypothesis testing.   

**P-value mining:** We discussed importance proper sampling in Chapter 9. Poor, or opportunistic, sampling can lead to false inferences with surprising ease. There are quite a few manifestations of this problem, which goes by many names, p-value mining, data dredging, and p-value fishing. These problems are particularly acute when dealing with large datasets.     

**False Discovery Rate Control:** The goal of hypothesis testing is to find important relationships in large complex datasets. These dataset typically contain a large number of variables. The **high-dimensional** nature of the data leads to some commonly encountered pitfalls of incorrect inference, **false discovery**. A related problem is cutting off a large-scale analysis when a desired relationship is 'found'. This practice of **p-value mining** often leads to unwarranted inferences. You will apply false discovery rate (FDR) control methods to address this problem.   

To get started, execute the code in the cell below to import the required packages. 

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
from scipy.stats import ttest_ind, f_oneway
from itertools import product, combinations
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import comb
import statsmodels.regression.linear_model as sm
import math
import time

pd.set_option("display.max_rows", None, "display.max_columns", None)

## Danger of P-Value Mining

There are many forms of **p-value mining**. There are many forms of this problem. For example, one temptation is to continue to collect data until the sample appears large enough to give a significant result. With a running example, we will explore the perils of this approach.   

The code in the cell below simulates two independently distributed random variables (no correlation) with a sample size of 700 each. Execute this code.  

In [ ]:
nr.seed(560)
covariance = np.array([[10.0,0.0],[0.0,5.0]])
uncorrelated_data = nr.multivariate_normal(mean=[0.0,0.0], cov=covariance, size=700)
uncorrelated_data

You can test the relationship between these two variables with a linear model. Execute the code in the cell below to compute a linear model of one variable regressed on the other and then display the summary. 

In [ ]:
uncorrelated_OLS_model = sm.OLS(uncorrelated_data[:,0],uncorrelated_data[:,1]).fit()
print(uncorrelated_OLS_model.summary())

As expected the slope coefficient shows no effect, as it is zero. You can see this by noticing that the confidence interval of the slope coefficient spans zero. Further, the adjusted R-squared value is essentially zero. 

Execute the code in the cell below and examine the resulting plot.

In [ ]:
data_frame = pd.DataFrame(uncorrelated_data, columns=['variable1','variable2'])

sns.set(color_codes=True)
g = sns.lmplot(x='variable1', y='variable2', data=data_frame, height=5, aspect = 1.2)

Examine the plot. The confidence interval of the best-fit regression line contains the zero effect (zero slope) case indicating that there is no relationship between these variables. 

Having confirmed that there is no significant relationship between the two variables, let's see what happens when we randomly sample from these data. The code in the cell below samples increasing numbers of cases (rows) from the data set, computes an ordinary least squares model for each sample, and appends the resulting p-value and slope coefficient to lists. Execute this code.

In [ ]:
indx_base = range(700)
slopes=[]
numbers=[]
p_values=[]
for i in range(6,500):
    temp_model = sm.OLS(uncorrelated_data[:i,0],uncorrelated_data[:i,1]).fit()
    slopes.append(temp_model._results.params[0])
    p_values.append(temp_model._results.pvalues[0])
    numbers.append(i)

We know that the actual slope coefficient should be close (indistinguishable) from zero. But, is the always the case as an increasing number of cases from the data set are sampled? To find out, execute the code in the cell below to plot the slope coefficient vs. the sample size. 

In [ ]:
## Plot slopes vs. sample size
fig, ax = plt.subplots(figsize=(7,5))
ax.plot(numbers, slopes, 'r')
ax.set_title('Estimated slope vs sample size') # Give the plot a main title
ax.set_xlabel('Sample size') # Set text for the x axis
ax.set_ylabel('slope') # Set text for y axis

The resulting plot has the look of a random walk. With increasing sample size the slope coefficient does approach the expected value of 0, there are some significant deviations along the way. In other words, random sampling of the data set can produce effects that might seem far from zero. 

What about the p-values of the slope coefficient? How does the apparent significance change as the data are randomly sampled? To find out, create and execute code in the cell below to plot the p-value vs. the sample size and examine the result. 

In [ ]:
## Plot p-values vs. sample size
fig, ax = plt.subplots(figsize=(7,5))
ax.plot(numbers, p_values, 'r')
ax.axhline(y=0.05, linestyle='dashed')
ax.set_title('Estimated p-values vs sample size') # Give the plot a main title
ax.set_xlabel('Sample size') # Set text for the x axis
ax.set_ylabel('P-values') # Set text for y axis

> **Exercise 15-1:** Examine these charts and answer the following questions:    
> 1. Do the paths of the slope coefficient and p-value resembles a random walk at small samples? 
> 2. As the sample size gets larger, does the slope coefficient and p-value converge to reasonable values?      
> 3. At some sample sizes do the p-values indicate possible false significance of the slope coefficient?

> **Answers:**     
> 1.   
> 2.      
> 3.     

## Multiple Hypothesis Tesing

Testing multiple hypothesis in high-dimensional data is problematic. Exhaustively testing all pairwise relationships between variables in a data set is a commonly used, but generally misleading, from of **multiple comparisons**. The chance of finding false significance, using such a **data dredging** approach, can be surprisingly high. 

To illustrate this problem we will perform multiple comparisons on **only 20 identically distributed independent (iid)** variables. Ideally, such tests should not find significant relationships, but the actual result is quite different. 

In this case, we will create a data set with 20 iid Normal distributions of 1000 samples each. Execute the code in the cell below to find this data and display the mean and variance of each variable.

In [ ]:
ncolumns = 20
nr.seed(234)
normal_vars = nr.normal(size=(1000,ncolumns))
print('The means of the columns are\n', np.mean(normal_vars, axis = 0))
print('\nThe variances of the columns are\n', np.var(normal_vars, axis = 0))

Notice that means are close to 0.0 and variances are close to 1.0 respectively. As expected, there is not much difference between these variables.

###  Testing the significance of differences 

With these variables generated, you will apply a [t-test](https://www.statisticshowto.com/probability-and-statistics/t-test/) to all pairwise combinations of identical Normally distributed variables. How many of these t-tests will show **significance** at the 0.05 cut-off level? With 20 variables there are 190 unique pairwise combinations. We expect to find a number of falsely significant test results from this many pairwise tests.  

> **Exercise 15-2:** You will now create and apply the following code for the reduce process:   
> 1. Define a function that takes the `normal_vars` array as an argument and returns a data frame with the following columns, i, j, t_statistic, p_value. You can use the following steps:     
>    - Instantiate a data frame of $n(n-1)/2$ rows and the columns specified above.     
>    - Using [itertools.combinations](https://docs.python.org/3/library/itertools.html#itertools.combinations) [enumerate](https://docs.python.org/3/library/functions.html#enumerate) over all combinatiions of variables in the `normal_vars` array.     
>    - Within the loop use [scipy.stats.ttest_ind](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) to compute the t-statistic and p-value for the variable pair.      
>    - Add the values of i, j, t-statistic and p-value to the data frame.   
> 2. Call the function, assigning the results to a data frame named `pairwise_tests`.
> 3. Execute the code and display the first 20 rows of the data frame.  

In [ ]:
## Put your code below









> 4. In the cell below, create a filter for pair test cases which are significant at the $\alpha=0.05$ level, and save these cases in a data frame named `significant_tests`. 
> 5. Print the number (len) of significant results.
> 6. Print the rows with the significant test results.

In [ ]:
## Put your code below   





> Notice the large number of apparently 'significant' test results. Answer the following questions:  
> 1. Is the number of false positive cases approximately what you expected?    
> 2. Examine which of the iid Normal vectors contribute to the false positive results. Are there vectors which contribute multiple times?   
> **End of exercise.**

>  **Answers:**   
> 1.         
> 2.    

### Bonferroni correction  

Several adjustments to the multiple comparisons problem have been proposed. Dunn published a method know as the **Bonferroni correction** in 1961. The Bonferroni correction is a widely used method to reduce the false positive rate of hypothesis tests.  The adjustment is simple:
$$\alpha_b = \frac{\alpha}{m}\\
with\\ 
m =\ number\ of\ tests$$

Can the Bonferroni correction help? Yes, by greatly increasing the confidence level required for a statistically significant result. The problem with the Bonferroni correction is the corresponding reduction in power as the cutoff value decreases. For big data problems with large numbers of groups, this issue can be especially serious. In fact, the Bonferroni correction can often lead to significant numbers of missed discoveries, or Type II errors. 

> **Exercise 15-3:** You will now apply the Bonferroni correction to determining the significance of the p-values of the pair-wise t-tests. Do the following:
> 1. Compute the Bonferroni significance level. 
> 2. Apply this cutoff to the p-values of all of the cases.
> 3. Print the number of false discoveries, or Type I errors. 
> Execute your code and examine the results

In [ ]:
## Put your code below 




> Does using the  Bonferroni correction reduce false significance tests, or even eliminate them?           

> **Answer:**      

> But, can we detect small effect with Bonferroni correction, as this method significantly reduces power of tests? Execute the code in the cell below, which compares a standard Normal to a Normal with a small mean (effect size), to find out. 

In [ ]:
nr.seed(567)
print("Bonferroni cutoff = {:6f}".format(significance_bonferroni))
t_statitic, p_value = ttest_ind(normal_vars[:,0], nr.normal(loc = 0.01, size=(1000,1)))
print("T-statistic = {0:4.2f}  and p-value = {1:4.2f}".format(t_statitic[0], p_value[0]))

> Provide a short answer to these questions:        
> 1. Given the Bonferroni correction and this difference in means, would this result be considered significant or would it be a type II error, and why?    
> 2. What does this result tell you about the downside of the Bonferroni correction as a FDR control method, in terms of finding significant results?      
> **End of exercise.**

> **Answers:**      
> 1.     
> 2.    

## Imporved False Discovery Rate Control Methods 

We have seen the potential pitfalls of multiple hypothesis testing. Further, we have seen that a simple approach to **false discovery rate (FDR) control** is not effective. You will now apply more sophisticated FDR control methods. 

The specific problem we will explore is to determine which genes lead to expression of a certain disease. In this example, there are gene expression data for 97 patients. Some of these patients have ulcerative colitis, a condition believed not to be inheritable, and others have Crohn's disease, which is believed to be genetically inherited.    

One approach to this problem is to perform hypothesis tests on the expression of the genes between patients with the two conditions. Since there are over 10,000 genes there is considerable chance for false discovery. Therefore, careful application of FDR control is required.

To continue with the example, execute the code in the cell below to load the data and print the dimensionality of the data frame. 

In [ ]:
gene_data = pd.read_csv('../data/ColonDiseaseGeneData-Cleaned.csv')
print(gene_data.shape)

There are data from 97 patients (rows) with 10,497 genes (features in columns). A large number of pairwise hypothesis tests are required!     

Execute the code in the cell below to view the first 5 columns of the data frame, which includes the expression of the first 4 genes.  

In [ ]:
 print(gene_data.iloc[:10,:5])      

You will apply two FDR control methods to these data to reduce the FDR rate, and compare the results to the Bonferroni method. These methods control the FDR, while not being overly conservative on true positive results, like the Bonferronic correction. 

### Holm's method

The first of these **Holm's method**. Holm's method operates on the ordered set of p-values, $D = \{ p_{(1)}, p_{(2)}, p_{(3)}, \ldots, p_{(n)} \}$. The threshold is dynamic with the order of the P-value. For the $ith$ p-value, $p(i)$ for $N$ pairwise tests is:  

$$p(i) \le Threshold(Holm's) = \frac{\alpha}{N - i + 1}$$

For example: for the 10th ordered p-value with 1,000 total tests (genes) and significance level of 0.05, the cutoff is:   

$$p(10) \le \frac{0.05}{1000 - 10 + 1} = 0.00005045$$

### Split the data frame

You will now use multiple t-tests to determine if there are significant differences in the gene expressions between patients with the different conditions. To do so you must separate the gene expressions for the two classes of patients.  

> **Exercise 15-4:** To start the processing of these data you will first create and execute code to create two new data frames, one for patients with each of the conditions, `ulcerative`, and `crohns`, by the following steps:   
> 1. Create a logical mask and group the values by `Disease State`. The mask is a boolean list, positive where the `Disease State` is 'Ulcerative Colitis (UC)'.
> 2. Use the mask to divide the original data frame into two data frames. Use the [numpy.logical_not](https://numpy.org/doc/stable/reference/generated/numpy.logical_not.html) for Chorns Disease.   
> 3. Return the transpose of the two data frames, removing the `Disease State` column. The result of this operation should be data frames with gene expressions for each patient in the columns and the gene identifier as the row index. Name the two data frames returned `ulcerative` and `crohns`.
> 4. Print the heads of the two data frames.  

In [ ]:
## Put your code below   
  
    
    
    



In [ ]:
ulcerative.head()

In [ ]:
crohns.head()

### Apply the t-test

> **Exercise 15-5:** With the key-value pairs organized by disease state, it is time to create and execute code of a reduce process. The reduce process will compute the pairwise t-statistics and p-values for each gene and return the sorted results. Specifically, your `gene_test` with arguments of the two mapped data frames will do the following:   
> 1. Create an empty results data frame with the index set to the index of the `ulcerative` data frame and columns named `t_statistic`, and `p-value`.
> 2. Use a `for` loop to iterate over the indicies of either data frame.  
> 3. Compute the t-statistic and p-value for the gene (key) for the pairwise tests between the data frames.
> 4. Assign the values to the row in the data frame using the gene identifier as index with the Pandas `loc` method.    
> 5. Use the [pandas.DataFrame.sort_values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) method to sort the results data frame, `inplace`, into ascending order.
> 6. Return the resulting data frame.
> 7. Call the function and print the first 10 rows of the data frame.  

In [ ]:
## Put your code bwlow   













### Significance of results 

With the gene data reduced to the t-test statistics, you will now determine the significance of these tests. It is important to understand that scientists believe that expression of a disease, like Corhn's, is only in a small number of genes, less than approximately 100.  

> **Exercise 15-6:** As a first step in understanding the gene expression significance complete and execute the code in the cell below to find the number of 'significant' genes using the simple single hypothesis test cutoff criteria.  

In [ ]:
significance_level =0.05
## Put your code below. 
significant_tests = gene_statistics[gene_statistics.iloc[:,1] < significance_level]
len(significant_tests)

> Does this large number of 'statistically significant' results appear credible, given that only a few genes are thought to have significant expression for this disease and why in terms of FDR?    
> **End of exercise.**     

> **Answer:**      

> **Exercise 15-7:** We have already seen that the Bonferroni correction is a rather conservative approach to testing the significance of large numbers of hypotheses. You will now use the Bonferroni correction to test the significance of the gene expression.  

In [ ]:
number_tests = len(gene_statistics)
significance_Bonferroni = significance_level/number_tests

## Put your code below. 




> Does the foregoing result seems reasonable in terms of discovery rate and FDR control?    
> **End of exercise.**   

> **Answer:**     

> **Exercise 15-08:** It may well be the case that the foregoing results using the Bonferroni correction is too conservative. You will now apply the Holms method to determining significance of the gene expression test results. In the cell below complete the `holms_significance` function with arguments of the results data frame and the significance level. This function does the following:  
> 1. Find the number of test results and compute the denominator used for the cutoff calculation. 
> 2. Compute the vector of thresholds using the Holms formula. Use the Python `range`function to get the values of the index i. But, keep in mind that range produces a zero-indexed iterator, and the algorithm needs a one-indexed list.  Use the [numpy.divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) function to perform the vector divide. Save these threshold values in a data frame in a 'holms_threshold' column.   
> 3. Using the threshold values compute a logical vector and save it in a logical type column named 'significance' in the data frame.
> 4. Return the data frame.
> Finally, execute the function and save the results in a data frame named `holms_results`. Then find and print the length of the subset where the 'significance' value is True. 

In [ ]:
## Put your code below









> Consider the differences between the calculation of the Bonferroni correction and Holm's threshold. Do you expect any practical difference in the number of significant tests given ordering by the p-value index, $i$? What is the meaning of the similarity or difference of the actual results in this case?  
> **End of exercise.**

> **Answer:**     

You can visualize the results of the Holm's method test. The plot has two key elements:  
1. Plot the curve of the p-values vs. the order number, i. The line is color coded by significance or not.
2. Plot the threshold line. This line is straight since the threshold is a linear function of i.

In [ ]:
def plot_significance(results, threshold):
    results['number'] = range(len(results))
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.lineplot(x='number',y=threshold, data=results, ax=ax, color='black', linewidth=0.5)
    sns.scatterplot(x='number',y='p_value', hue='significant', data=results, s=3, ax=ax)
    ax.set_title('Significance of gene expression')
    ax.set_xlabel('Gene number')
    ax.set_ylabel('p-value')
    
plot_significance(holms_results.iloc[:150,:].copy(), 'holms_threshold')    

Notice the following about this plot:  
1. The p-value significance line crosses the threshold point at an apparent break point.   
2. The significant p-values are all very small since there are so many tests.
3. The Holm's threshold is does not change very much over the first 150 test results. 

### Benjamini-Hochberg FDR Control 

The Benjamini-Hochberg FDR control algorithm is another way to control false discoveries. Stat with an ordered set of $n$ p-values, $D = \{ p_{(1)}, p_{(2)}, p_{(3)}, \ldots, p_{(n)} \}$ we define a false discovery rate, $q$:

$$FDR(D) \le q$$

The cutoff threshold for the ith p-value is then:
$$p_{(i)} \le Threshold(D_q) = \frac{q}{n} i$$

> **Exercise 15-9:** In this exercise you will apply the Benjamini-Hochberg FDR control algorithm for testing the significance of the gene expressions. The `BH_significance` function is quite similar to the Holm's method function you have already created. Given the large number of genes you must use a low false discovery rate, $q = 0.001$, or 1 out of 1,000. 
> Execute your function, saving the result in a data frame named `BH_results`. Then print the number of significant cases. 

In [ ]:
## Put you code below









> This result differs from the first two FDR control methods you have applied. Given the false discovery parameter of 0.001 do you think this is a reasonable result and why? 

> **Answer:**     

Finally, execute the code in the cell below and examine the resulting plot.

In [ ]:
plot_significance(BH_results.iloc[:150,:].copy(), 'bh_threshold')    

> **Exercise 15-10**: Compare the plots of results of the Benjamini-Hochberg FDR control method to those of Holm's method. Why does the Benjamini-Hochberg FDR control method give a more dynamic or adaptive result for the case where $i << n$? 

> **Answer:**     

##### Copyright 2021, 2022, 2023, Stephen F. Elston. All rights reserved. 